In [1]:
import os
import numpy as np
import pandas as pd
import lttb 

import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from collections import OrderedDict
from dash.dependencies import Input, Output
import datashader as ds
import datashader.transfer_functions as tf

import django
django.setup()

from antennas.models import AntennaStatus


In [2]:
last_stat = AntennaStatus.objects.last()

In [3]:
stats = AntennaStatus.objects.filter(time=last_stat.time).order_by("antenna")

In [5]:
# freqs = np.linspace(50,250, 2048) * 1e6
# spectra = np.random.uniform(.1,1, 2048)

In [52]:
df = pd.DataFrame()
df_sampled = pd.DataFrame()
for stat in stats:
#     _freqs = np.asarray(freqs) / 1e6
#     _spectra = 10 * np.log10(np.ma.masked_invalid(spectra)).filled(None)
    _freqs = np.asarray(stat.frequencies) / 1e6
    _spectra = 10 * np.log10(np.ma.masked_invalid(stat.spectra)).filled(None)
    
    data = [{
        "freqs": f,
        "spectra": d,
        "antpol": f"{stat.antenna.ant_number}{stat.antenna.polarization}"
    }
        for f, d in zip(_freqs, _spectra)
    ]
    df1 = pd.DataFrame(data)
    df = df.append(df1)
    
    downsampled = lttb.downsample(
        np.stack(
            [
                _freqs, 
                _spectra,
            ], 
            axis=1
        ),
        np.round(_freqs.size / 5.).astype(int) if _freqs.size/5 > 3 else 3
    )
    data1 = [{
        "freqs": f,
        "spectra": d,
        "antpol": f"{stat.antenna.ant_number}{stat.antenna.polarization}"
    }
        for f, d in zip(downsampled[:,0], downsampled[:,1])
    ]
    df1 = pd.DataFrame(data1)
    df_sampled = df_sampled.append(df1)

In [53]:
fig = go.Figure()
for antpol in df.antpol.unique():
    df1 = df[df.antpol == antpol]
    dfs1 = df_sampled[df_sampled.antpol == antpol]
    trace = go.Scatter(
        x=df1.freqs,
        y=df1.spectra,
        name=antpol,
        mode="lines",
#         line_shape="spline",
        hovertemplate="%{x:.1f}\tMHz<br>%{y:.3f}\t[dB]",
    )
    
    t2 = go.Scatter(
        x=dfs1.freqs,
        y=dfs1.spectra,
        name=f"{antpol}-lttb",
        mode="lines",
        hovertemplate="%{x:.1f}\tMHz<br>%{y:.3f}\t[dB]",
    )
# fig = df.plot(x="freqs", y="spectra", label="antpol")
fig.update_layout(
        xaxis={"title": "Frequency [MHz]"},
        yaxis={"title": "Power [dB]"},
        title={
            "text": "Autocorrelations",
            "xref": "paper",
            "x": 0.5,
            "font": {"size": 24},
        },
        autosize=True,
        showlegend=True,
        legend={"x": 1, "y": 1},
        margin={"l": 40, "b": 30, "r": 40, "t": 46},
        hovermode="closest",
    )

fig.add_trace(trace)
fig.add_trace(t2)